## Notebook for Fine-Tuning the YOLOv11n on Customs Datasets

In [ ]:
import cv2
import matplotlib.pyplot as plt
import glob
import random
import os

In [ ]:
# Define global variables for image and label paths with ID 1 for camera 1 (& 2 for camera 2)
cam_id = 2 
DATASET_PATH = f"../data/camera_{cam_id}"
TRAIN_IMAGE_PATH = DATASET_PATH + "/train/images/"
TRAIN_LABEL_PATH = DATASET_PATH + "/train/labels/"

## Visualize Images from the Dataset

In [ ]:
def yolo2bbox(bbox):
    """
    Convert YOLO format bounding box to (xmin, ymin, xmax, ymax).

    Args:
        bbox (list or tuple): [x_center, y_center, width, height] (all normalized).

    Returns:
        tuple: (xmin, ymin, xmax, ymax) (all normalized).
    """
    xmin = bbox[0] - bbox[2] / 2
    ymin = bbox[1] - bbox[3] / 2
    xmax = bbox[0] + bbox[2] / 2
    ymax = bbox[1] + bbox[3] / 2
    return xmin, ymin, xmax, ymax

In [ ]:
def plot_box(image, bboxes):
    """
    Draw bounding boxes on the image.

    Args:
        image (np.ndarray): The input image.
        bboxes (list): List of bounding boxes in YOLO format [x_center, y_center, width, height].

    Returns:
        np.ndarray: Image with bounding boxes drawn.
    """
    h, w, _ = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        xmin = int(x1 * w)
        ymin = int(y1 * h)
        xmax = int(x2 * w)
        ymax = int(y2 * h)

        thickness = max(2, int(w / 275))

        cv2.rectangle(
            image, (xmin, ymin), (xmax, ymax), color=(0, 0, 255), thickness=thickness
        )
    return image

In [ ]:
def plot(image_paths, label_paths, num_samples):
    """
    Visualize random samples from the dataset with bounding boxes.

    Args:
        image_paths (str): Path to the directory containing images.
        label_paths (str): Path to the directory containing label files.
        num_samples (int): Number of samples to visualize.

    Returns:
        None
    """
    all_images = []
    all_images.extend(glob.glob(image_paths + "/*.jpg"))
    all_images.sort()
    num_images = len(all_images)

    print(f"Number of images found: {num_images}")

    if num_images == 0:
        print(f"No images found in the directory: {image_paths}")
        return

    plt.figure(figsize=(15, 12))
    for i in range(min(num_samples, num_images)):
        j = random.randint(0, num_images - 1)
        image_path = all_images[j]
        image_name = ".".join(image_path.split(os.path.sep)[-1].split(".")[:-1])
        image = cv2.imread(image_path)

        if image is None:
            print(f"Failed to load image: {image_path}")
            continue

        label_file = os.path.join(label_paths, image_name + ".txt")

        if not os.path.exists(label_file):
            print(f"Label file not found: {label_file}")
            continue

        try:
            with open(label_file, "r") as f:
                bboxes = []
                labels = []
                label_lines = f.readlines()
                for line_num, label_line in enumerate(label_lines, 1):
                    parts = label_line.strip().split()
                    if len(parts) != 5:
                        raise ValueError(
                            f"Unexpected format in {label_file}, line {line_num}: {label_line.strip()}"
                        )

                    label, x_c, y_c, w, h = parts
                    x_c, y_c, w, h = map(float, (x_c, y_c, w, h))
                    bboxes.append([x_c, y_c, w, h])
                    labels.append(label)

            result_image = plot_box(image, bboxes)
            plt.subplot(2, 2, i + 1)
            plt.imshow(result_image[:, :, ::-1])
            plt.axis("off")
        except Exception as e:
            print(f"Error processing file: {image_path}")
            print(f"Corresponding label file: {label_file}")
            print(f"Error: {str(e)}")
            continue

    plt.subplots_adjust(wspace=1)
    plt.tight_layout()
    plt.show()
    return None

In [ ]:
# Visualize a few training images.
plot(
    image_paths=TRAIN_IMAGE_PATH,
    label_paths=TRAIN_LABEL_PATH,
    num_samples=4,
)

## Dataset YAML File

In [ ]:
def write_yolov11n_config(dataset_path, filepath):
  """
  Write a YOLOv11n dataset config YAML file.

  Args:
    dataset_path (str): Path to the dataset root.
    filepath (str): Output YAML filename.
  """
  content = f"""path: {dataset_path}
train: 'train/images'
val: 'valid/images'

# class names
names:
  0: 'Riped berries'
"""
  with open(filepath, "w") as f:
    f.write(content)

In [ ]:
DATA_CONFIG = f"yolov11n_config_camera_{cam_id}.yaml"
write_yolov11n_config(DATASET_PATH, DATA_CONFIG)

## YOLOv11 Nano Fine-Tuning

In [ ]:
# Define fine-tuning parameters
EPOCHS = 5
BATCH_SIZE = 8
IMAGE_SIZE = 1280
BASE_MODEL = "yolo11n.pt"
BEST_MODEL = f"runs/detect/camera_{cam_id}/yolov11n_train/weights/best.pt"

In [ ]:
!yolo task = detect \
    mode = train \
    model = {BASE_MODEL} \
    imgsz = {IMAGE_SIZE} \
    epochs = {EPOCHS} \
    batch = {BATCH_SIZE} \
    data = {DATA_CONFIG} \
    name = "camera_{cam_id}/yolov11n_train"

## Evaluation on Validation Images

In [ ]:
!yolo task = detect \
    mode = val \
    model = {BEST_MODEL} \
    data = {DATA_CONFIG} \
    name = "camera_{cam_id}/yolov11n_eval"

## Inference on Validation Images

In [ ]:
!yolo task = detect \
    mode = predict \
    model = {BEST_MODEL} \
    source = "../data/camera_{cam_id}/valid/images" \
    imgsz = {IMAGE_SIZE} \
    name = "camera_{cam_id}/yolov11n_inference" \
    hide_labels = True

## Visualize Validation Results

In [ ]:
def visualize(result_dir, num_samples=4):
    """
    Function accepts a directory of images and plots
    them in a 2x2 grid.

    Args:
        result_dir (str): Path to the directory containing result images.
        num_samples (int): Number of images to visualize (default is 4).

    Returns:
        None
    """
    plt.figure(figsize=(20, 12))
    # Remove leading slash if present
    result_dir = result_dir.lstrip("/")
    image_names = glob.glob(os.path.join(result_dir, "*.jpg"))
    if not image_names:
        print(f"No images found in {result_dir}")
        return
    random.shuffle(image_names)
    for i, image_name in enumerate(image_names[:num_samples]):
        image = plt.imread(image_name)
        plt.subplot(2, 2, i + 1)
        plt.imshow(image)
        plt.axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
visualize(f"/runs/detect/camera_{cam_id}/yolov11n_inference/")